In [4]:
import yaml
import json
import os


### Search Tool

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
with open("config/config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

In [7]:
with open(config['tavily_api_key_path'], 'r') as tavily_key_file:
    os.environ["TAVILY_API_KEY"] = json.load(tavily_key_file)['tavily']

search = TavilySearchResults()

### Use Gemni Pro LLM to Built an Agent

### LangSmith

In [58]:
from langsmith import traceable

In [59]:
with open("secrets/config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

os.environ["LANGCHAIN_TRACING_V2"] = config['langsmith']['langchain_tracing_v2']
os.environ["LANGCHAIN_ENDPOINT"] = config['langsmith']['langchain_endpoint']
os.environ["LANGCHAIN_PROJECT"] = config['langsmith']['langchain_project']
os.environ["LANGCHAIN_API_KEY"] = config['langsmith']['langchain_api_key']

#### LLM

In [65]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro")

In [56]:
@traceable
def invoke_llm(query):
    return llm.invoke(query)

In [57]:
invoke_llm('Who is current president of USA?')

AIMessage(content='Joe Biden is the current president of the United States. He was inaugurated on January 20, 2021, and is the 46th president of the United States.', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'usage_metadata': {'prompt_token_count': 7, 'candidates_token_count': 39, 'total_token_count': 46}}, id='run-99bf8e9d-b536-4176-a584-0c9bf872221e-0')

### Define Tool

In [69]:
from langchain.agents import tool

@tool
@traceable
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [70]:
@traceable
def create_tools():
    return [get_word_length]

tools = create_tools()


#### Create Prompt

In [73]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

@traceable
def create_prompt():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are very powerful assistant, but don't know current events",
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    return prompt


In [74]:
prompt = create_prompt()

#### Bind Tools

In [71]:
llm_with_tools = llm.bind_tools(tools)

#### Create Agent

In [75]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

@traceable
def create_agent():
    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent

In [76]:
agent = create_agent()

#### Execute the Agent

In [77]:
from langchain.agents import AgentExecutor

@traceable
def create_agent_executor():
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    return agent_executor

In [78]:
agent_executor = create_agent_executor()

In [79]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The length of the word "eudca" is 5 letters. 
A letter is a written symbol that represents a sound in a language. 
The word "eudca" has 5 letters: e, u, d, c, and a.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments': '{"word": "eudca"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5a03b419-cca5-4d5d-b917-f9a0720c40d8', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'c9fa83a9-e419-4816-95e5-0daac0b7285d'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word": "eudca"}', 'id': 'c9fa83a9

# LangSmith

In [61]:
from langsmith import traceable
from openai import OpenAI


In [62]:
with open("secrets/config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

os.environ["LANGCHAIN_TRACING_V2"] = config['langsmith']['langchain_tracing_v2']
os.environ["LANGCHAIN_ENDPOINT"] = config['langsmith']['langchain_endpoint']
os.environ["LANGCHAIN_PROJECT"] = config['langsmith']['langchain_project']
os.environ["LANGCHAIN_API_KEY"] = config['langsmith']['langchain_api_key']


In [64]:
with open('secrets/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

client = OpenAI(organization=config['open_ai']['organization'],
                project=config['open_ai']['project_id'],
                api_key=config['open_ai']['api_key'])

In [49]:
# This is the retriever we will use in RAG
# This is mocked out, but it could be anything we want
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
@traceable
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:
    
    {docs}""".format(docs="\n".join(docs))
    
    return client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-3.5-turbo",
    )


In [50]:
rag("where did harrison work")

ChatCompletion(id='chatcmpl-9TN956xCLXK8V0tQnZxNiLbvUZwcQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Harrison worked at Kensho.', role='assistant', function_call=None, tool_calls=None))], created=1716787511, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=34, total_tokens=41))